In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os

In [26]:
def get_csv_data(path_logs, path_csv):
    data = pd.read_csv(path_csv, sep='\t')
    dfs = []
    for dirname, _, filenames in os.walk(path_logs):
        for filename in filenames:
            if filename.endswith('.csv'):
                file_path = os.path.join(dirname, filename).replace('\\', '/')
                df = pd.read_csv(file_path,sep=' ')
                
                row = data.loc[data['log_id'] == int(filename[:-4])] # https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
                # print(row)
                arr = row.values.tolist()
                arr.append(df)
                dfs.append(arr)
    return dfs

In [27]:
dfs = get_csv_data('../the-attentive-cursor-dataset/logs', '../the-attentive-cursor-dataset/groundtruth.tsv')

In [28]:
cleandf = [] #note + liste des inputs
for row in range(len(dfs)):
    cleandf.append([dfs[row][0][2],dfs[row][1]['event'].value_counts()])

In [29]:

tab = np.array(cleandf, dtype='object')
for row in tab :
    for input in ['mouseup','focus','load','beforeunload','unload','touchend'] : #liste arbitraire des actions à enlever
        if input in row[1].keys() :
            row[1].pop(input)

#liste des actions à conserver
all_inputs = []
for row in tab :
    for input in row[1].keys() :
        if input not in all_inputs :
            all_inputs.append(input)

In [30]:
#normalisation v1
"""
for input in all_inputs :
    tmp = []
    for row in tab :
        if input in row[1].keys() :
            tmp.append(row[1].get(input))
    mean = np.mean(tmp)
    for row in tab :
        if input in row[1].keys() :
            row[1][input] = row[1][input]/mean

print(tab)
"""


'\nfor input in all_inputs :\n    tmp = []\n    for row in tab :\n        if input in row[1].keys() :\n            tmp.append(row[1].get(input))\n    mean = np.mean(tmp)\n    for row in tab :\n        if input in row[1].keys() :\n            row[1][input] = row[1][input]/mean\n\nprint(tab)\n'

In [31]:
tmp_all_x = [] #stock temporairement les données d'entrées
tmp_all_y = [] #stock temporairement les données de sortie

for row in tab :
    tmp_all_x.append([])
    for input in all_inputs :
        if input in row[1].keys() :
            tmp_all_x[len(tmp_all_x)-1].append(row[1].get(input))
        else :
            tmp_all_x[len(tmp_all_x)-1].append(0)
    tmp_all_y.append(row[0])

all_x = np.array(tmp_all_x) #données en entrée du modèle
all_y = np.array(tmp_all_y) #données en sortie du modèle
print(all_x)

[[61 22  9 ...  0  0  0]
 [ 7  9  3 ...  0  0  0]
 [11 39  3 ...  0  0  0]
 ...
 [ 0  8  4 ...  0  0  0]
 [ 0  8  2 ...  0  0  0]
 [ 0  4  2 ...  0  0  0]]


In [40]:
#normalisation v2 (la v1 est un peu mieux)
"""
for i in range(len(all_inputs)) :
    print(i)
    tmp = []
    for row in all_x :
        tmp.append(row[i])
    mean = np.mean(tmp)
    for row in all_x :
        row[i] = row[i]/mean
    
print(all_x)
"""

0
1
2
3
4
5
6
7
8
9
10
11
12
13
[[21  1  3 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 [ 2  3  0 ...  0  0  0]
 ...
 [ 0  0  1 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [41]:

inputs = keras.Input(shape=(len(all_inputs),))
x = tf.keras.layers.Dense(1, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(6, activation=tf.nn.softmax)(x)

model = keras.Model(inputs=inputs, outputs = outputs)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)



In [42]:
history = model.fit(all_x, all_y, epochs=20, validation_split=0.1)

Epoch 1/20
44/44 [==============================] - 1s 9ms/step - loss: 1.8247 - accuracy: 0.2139 - val_loss: 1.8102 - val_accuracy: 0.2387
Epoch 2/20
44/44 [==============================] - 0s 4ms/step - loss: 1.8153 - accuracy: 0.2534 - val_loss: 1.8011 - val_accuracy: 0.2516
Epoch 3/20
44/44 [==============================] - 0s 4ms/step - loss: 1.8065 - accuracy: 0.2670 - val_loss: 1.7932 - val_accuracy: 0.2710
Epoch 4/20
44/44 [==============================] - 0s 4ms/step - loss: 1.7986 - accuracy: 0.2828 - val_loss: 1.7853 - val_accuracy: 0.3226
Epoch 5/20
44/44 [==============================] - 0s 4ms/step - loss: 1.7910 - accuracy: 0.2965 - val_loss: 1.7783 - val_accuracy: 0.3226
Epoch 6/20
44/44 [==============================] - 0s 4ms/step - loss: 1.7837 - accuracy: 0.3130 - val_loss: 1.7718 - val_accuracy: 0.3226
Epoch 7/20
44/44 [==============================] - 0s 6ms/step - loss: 1.7768 - accuracy: 0.3144 - val_loss: 1.7653 - val_accuracy: 0.3226
Epoch 8/20
44/44 [==